## Usando una API para obtener información

In [1]:
import requests
from getpass import getpass
from time import sleep
import os 
import hashlib

USERNAME = "Ok-Tailor2962" 
PASSWORD = "SMB03042001c#"

CLIENT_ID = "HUMRGABjM_tzXiuGPbTFRw" 
CLIENT_SECRET = "3UTcnYjGifMuDI-No_FA2TnjFEacnQ"
USER_AGENT = "Mozilla/5.0 (platform; rv:geckoversion) Gecko/geckotrail Firefox/firefoxversion"

def login(username, password):
    if password is None:
        password = getpass.getpass("Enter reddit password for user {}: ".format(username))    
    headers = {"User-Agent": USER_AGENT}
    # Setup an auth object with our credentials
    client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
    # Make a post request to the access_token endpoint
    post_data = {"grant_type": "password", "username": username, "password": password}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth,     
                             data=post_data, headers=headers) 
    return response.json()

token = login(USERNAME, PASSWORD)
print(token)

{'access_token': '13172944410664-N7EdV_6rRIdac0_bwZ_Iw3nVIgtQVA', 'token_type': 'bearer', 'expires_in': 86400, 'scope': '*'}


## Redit como un recurso de información

In [2]:
subreddit = "worldnews"

url = "https://oauth.reddit.com/r/{}".format(subreddit)

headers = {"Authorization": "bearer {}".format(token['access_token']), "User-Agent": USER_AGENT}

response = requests.get(url, headers=headers)

result = response.json()
for story in result['data']['children']: 
    print(story['data']['title'])

/r/WorldNews Live Thread: Russian Invasion of Ukraine Day 386, Part 1 (Thread #527)
France's President Macron overrides parliament to pass retirement age bill
US military releases video from Russian fighter jet crash with drone
Russian leadership approved actions of jets that damaged U.S. drone, officials say
Putin tells Russia's billionaires to put patriotism before profit
Russia has committed 'wide range' of war crimes in Ukraine, U.N inquiry finds
Poland to transfer MiG-29 jets to Ukraine within days – DW – 03/16/2023
Moldova no longer needs Russian gas, minister says
Meta to end Canadians’ access to news on Facebook and Instagram if Bill C-18 becomes law
The 8-year-old daughter of a Vladimir Putin ally is listed as the owner of a multimillion-dollar apartment in one of London's most exclusive areas
‘Hostile, authoritarian’ UK downgraded in civic freedoms index
Russia's forced transfer and deportation of Ukrainian children to areas under its control amounts to a war crime, UN invest

## Obteniendo los datos

In [3]:
def get_links(subreddit, token, n_pages=5):
    stories = []
    after = None
    for page_number in range(n_pages):
        # Sleep before making calls to avoid going over the API limit
        sleep(2)
        # Setup headers and make call, just like in the login function
        headers = {"Authorization": "bearer {}".format(token['access_token']), "User-Agent": USER_AGENT} 
        url = "https://oauth.reddit.com/r/{}?limit=100". format(subreddit)
        if after:
            # Append cursor for next page, if we have one
            url += "&after={}".format(after)
        response = requests.get(url, headers=headers)
        result = response.json()
        # Get the new cursor for the next loop
        after = result['data']['after']
        # Add all of the news items to our stories list
        for story in result['data']['children']:
            stories.append((story['data']['title'], story['data']['url'], story['data']['score']))
    return stories

stories = get_links("worldnews", token)

print(stories)

[('/r/WorldNews Live Thread: Russian Invasion of Ukraine Day 386, Part 1 (Thread #527)', 'https://www.reddit.com/live/18hnzysb1elcs', 1301), ("France's President Macron overrides parliament to pass retirement age bill", 'https://www.cnbc.com/2023/03/16/frances-macron-overrides-parliament-to-pass-pension-reform-bill.html', 16124), ('US military releases video from Russian fighter jet crash with drone', 'https://abcnews.go.com/US/us-military-releases-video-drone-showing-collision-russian/story?id=97902649', 33821), ('Russian leadership approved actions of jets that damaged U.S. drone, officials say', 'https://news.yahoo.com/russian-leadership-approved-actions-jets-001937661.html', 3059), ("Putin tells Russia's billionaires to put patriotism before profit", 'https://www.reuters.com/world/europe/putin-urges-russias-billionaires-invest-face-sanctions-war-2023-03-16/', 1621), ("Russia has committed 'wide range' of war crimes in Ukraine, U.N inquiry finds", 'https://www.reuters.com/world/euro

In [3]:
data_folder = os.path.join(os.path.expanduser("~"), "Data", "websites", "raw")

## Encontrar las historias en sitios web arbitrarios

In [ ]:

number_errors = 0
for title, url, score in stories[414:len(stories)]:
#for title, url, score in stories:
    output_filename = hashlib.md5(url.encode()).hexdigest() 
    fullpath = os.path.join(data_folder, output_filename + ".txt")
    try: 
        response = requests.get(url) 
        data = response.text 
        with open(fullpath, 'w') as outf: 
            outf.write(data)
        print("Successfully completed {}".format(title))
    except Exception as e:
        number_errors += 1
        # You can use this to view the errors, if you are getting too many:
        # raise
print("Successfully all completed")

Successfully completed Volkswagen to open first North American EV battery plant in St. Thomas, Ont. | CBC News
Successfully completed /r/WorldNews Live Thread: Russian Invasion of Ukraine Day 383, Part 1 (Thread #524)
Successfully completed British-led design chosen for AUKUS submarine project
Successfully completed Former members of Reuzegom fraternity at KU Leuven, formerly the Catholic University of Leuven, back in Belgian court over death of black student Sanda Dia during hazing
Successfully completed Latest Transatlantic Narco-Submarine In Europe Has Same Builder As Last One
Successfully completed F-16s will 'eventually' be sent to Ukraine, former U.S. Joint Chiefs of Staff chair says
Successfully completed Moldova police say they foiled Russia-backed unrest plot
Successfully completed China to resume issuing all visa types for first time since 2020


## Extrayendo contenido (autoria propia)

In [4]:
from bs4 import BeautifulSoup

#Eliminación de texto entre una etiqueta html (<math></math>, <verbose></verbose>, etc)
def removeBetweenTag(tagR, actI, txtF):
    wordAux = ''
    actI+=1
    
    initI = actI-1
    #Se itera hasta encontrar la etiqueta de cierre </+etiqueta
    while wordAux!=('</'+tagR):
        #Si se llega al final del texto se lanza un mensaje de error
        if actI>=len(txtF):
            print('Se encontró un < sin cerrar')
            print('Caracter empezo en '+ str(initI))
            break
        auxC = txtF[actI]
        if auxC == '<':
            initII = actI
            while auxC != '>':
                if actI>=len(txtF):
                    print('Se encontró un < sin cerrar')
                    print('Caracter empezo en '+ str(initII))
                    break

                wordAux += auxC
                actI+=1
                auxC = txtF[actI]
        else:
            wordAux=''
            actI+=1
    return actI

#Eliminación de cierto caracter que se repite hasta un "numb" de veces
def removeRepite(text, chrRmv, numb):
    chrFind = chrRmv+chrRmv+chrRmv
    for _ in range(0, numb-3):
        chrFind+=chrRmv
        text.replace(chrFind, '')
    return text

#Eliminación de texto entre un caracter ("dummy text", /html comment/, etc)
def removeBetweenChr(chrR, actI, txtF):
    auxC=''
    initI = actI
    while auxC != chrR:
        actI+=1
        auxC = txtF[actI]
        if actI>len(txtF):
            print('Falta '+chrR+' de cierre')
            print('Caracter empezo en '+ str(initI))
            break
    return actI

#Eliminación de texto repetido
def removeRepiteChr(chrR, txtF):
    auxC=''
    i = 0
    newTxt=''
    while i<(len(txtF)-2):
        if i>len(txtF):
            break
        auxC = txtF[i]

        if(auxC == chrR):
            while txtF[i+1] == chrR:
                i+=1
                if i<len(txtF):
                    break
        newTxt += auxC
        i+=1
    return newTxt

#Obtencion y clasificacion de las palabras obtenidas en una url
def getText(filePath):
    try:
        #Caracteres y etiquetas html a eliminar
        uslessChars = [':', ",", ".", "/", "+", "\"", "\'", "\\", "\n", "$", "-", ">", '[', "]", "\t", "#"]
        uslessTags = ['script', 'semmantics', 'math', 'annotation', 'style']
        mystr = open(filePath).read()
        
        newStr = ''
        try:
            auxC = ''
            
            i = 0
            wordAux=''
            #Se itera entre el codigo obtenido y obtener solo el texo
            while i<len(mystr):
                auxC = mystr[i]

                #Se eliminan caracteres y etiquetas de html
                if(auxC == '<'):
                    initI = i
                    while auxC != '>':
                        i+=1
                        auxC = mystr[i]
                        wordAux+=auxC
                        if i>len(mystr):
                            print('Se encontró un < sin cerrar')
                            raise
                        
                        #Si se detecta que es una de las etiquetas sin texto util entre ellas
                        # se llama a la funcion que elimina ese texto y las etiquetas
                        if wordAux in uslessTags:
                            i = removeBetweenTag(wordAux, i, mystr)
                            break
                            
                wordAux=''
                    
                #Si se detecta que es un de los caracteres sin texto util entre ellos
                # se llama a la funcion que elimina ese texto
                if(auxC == '['):
                    i = removeBetweenChr(']', i, mystr)

                if(auxC == '{'):
                    i = removeBetweenChr('}', i, mystr)
                
                if(auxC != '\n'):
                    newStr += ''+auxC
                
                i+=1
        except:
            try:
                soup = BeautifulSoup(mybytes, features="html.parser")
                for script in soup(["script", "style"]):
                    script.extract()
                    
                text = soup.get_text()
                lines = (line.strip() for line in text.splitlines())
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                mystr = '\n'.join(chunk for chunk in chunks if chunk)
            except:
                soup = BeautifulSoup(mybytes, features="html.parser", from_encoding="iso-8859-1")
                for script in soup(["script", "style"]):
                    script.extract()
                    
                text = soup.get_text()
                lines = (line.strip() for line in text.splitlines())
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                mystr = '\n'.join(chunk for chunk in chunks if chunk)
                
        #Se eliminan los caracteres que no son utiles
        for ch in uslessChars:
            newStr = newStr.replace(ch, " ")
        newStr = removeRepiteChr(' ', newStr)
        newStr = removeRepiteChr('\n', newStr)

        newStr = removeRepite(newStr, 't', 75)
        newStr = removeRepite(newStr, 'e', 25)
    #Si ocurre algun error se guarda ''
    except Exception as e:
        print('Error en la peticion: '+str(e))
        newStr = ''
    return newStr

## Extrayendo el contenido

In [5]:
from lxml import etree

filenames = [os.path.join(data_folder, filename) for filename in os.listdir(data_folder)]
text_output_folder = os.path.join(os.path.expanduser("~"), "Data", "websites", "textonly")
skip_node_types = ["script", "head", "style", etree.Comment]

parser = etree.HTMLParser()

def get_text_from_file(filename):
    inf = open(filename)
    if len(inf.read()) == 0:
        return ""
        
    html_tree = etree.parse(inf, parser)
    return get_text_from_node(html_tree.getroot())

def get_text_from_node(node):
    try:
        if len(node) == 0: 
            # No children, just return text from this item
            if node.text: 
                return node.text 
            else:
                return ""
        else:
            # This node has children, return the text from it:
            results = (get_text_from_node(child) for child in node if child.tag not in skip_node_types)
        result = str.join("n", (r for r in results if len(r) > 1))
        print(result)
        
        if len(result) >= 100:
            return result
        else:
            return ""
    except:
        return ""

for filename in os.listdir(data_folder):
    text = get_text_from_file(os.path.join(data_folder, filename))
    if text == "":
        text = getText(os.path.join(data_folder, filename))
    
    with open(os.path.join(text_output_folder, filename), 'w') as outf: 
        outf.write(text)

print('Finished')

Se encontró un < sin cerrar
Caracter empezo en 107236
Se encontró un < sin cerrar
Caracter empezo en 112868
Finished
